In [5]:
# سلول اول – تنظیم محیط و وارد کردن کتابخانه‌های پایه

import os
import sys
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, confusion_matrix

# تنظیم مسیر پروژه و src
project_root = r"C:\Users\HsH-HsH\weighted_split_criteria"
src_path = os.path.join(project_root, "src")

# اضافه کردن src به مسیر پایتون
if src_path not in sys.path:
    sys.path.append(src_path)

print("✅ محیط تنظیم شد:")
print(f"   project_root = {project_root}")
print(f"   src_path     = {src_path}")


✅ محیط تنظیم شد:
   project_root = C:\Users\HsH-HsH\weighted_split_criteria
   src_path     = C:\Users\HsH-HsH\weighted_split_criteria\src


In [6]:
# سلول دوم – بارگذاری دیتاست

# مسیر فایل dataset
dataset_path = os.path.join(project_root, "sample_dataset_30_numeric.csv")

# بارگذاری داده‌ها
df = pd.read_csv(dataset_path)

# نمایش شکل و ستون‌ها
print("📊 شکل dataset:", df.shape)
print("📋 ستون‌ها:", list(df.columns))
print("\n📈 نوع داده‌ها:")
print(df.dtypes)

# نمایش پنج سطر اول
print("\n📋 نمونه از داده‌ها:")
print(df.head())


📊 شکل dataset: (30, 8)
📋 ستون‌ها: ['Age', 'LastTransaction', 'Churn', 'Gender_female', 'Gender_male', 'Payment Method_cash', 'Payment Method_cheque', 'Payment Method_credit card']

📈 نوع داده‌ها:
Age                           int64
LastTransaction               int64
Churn                         int64
Gender_female                 int64
Gender_male                   int64
Payment Method_cash           int64
Payment Method_cheque         int64
Payment Method_credit card    int64
dtype: object

📋 نمونه از داده‌ها:
   Age  LastTransaction  Churn  Gender_female  Gender_male  \
0   38              171      1              0            1   
1   24              140      1              1            0   
2   82               39      0              0            1   
3   34              217      1              0            1   
4   70              170      0              1            0   

   Payment Method_cash  Payment Method_cheque  Payment Method_credit card  
0                    0          

In [7]:
# سلول سوم – آماده‌سازی Features و Target

# تعریف متغیر هدف (Churn)
y = df['Churn'].values
print(f"📊 Target (Churn): {y}")
print(f"📈 توزیع کلاس‌ها: {np.bincount(y)}")
print(f"📊 درصد هر کلاس: Class 0: {(y==0).sum()/len(y)*100:.1f}%, Class 1: {(y==1).sum()/len(y)*100:.1f}%")

# تعریف Features (تمام ستون‌ها به جز Churn)
feature_columns = [col for col in df.columns if col != 'Churn']
X = df[feature_columns].values
print(f"\n🔧 Features: {feature_columns}")
print(f"📊 شکل X: {X.shape}")
print(f"📊 شکل y: {y.shape}")

# نمایش نمونه‌ها
print("\n📋 نمونه از X (اولین 3 سطر):")
for i in range(3):
    print(f"   سطر {i}: {X[i]}")

print("\n📋 نمونه از y (اولین 10 مقدار):")
print(f"   {y[:10]}")

# بررسی نهایی
n_samples = len(X)
n_features = len(feature_columns)
n_classes = len(np.unique(y))
print(f"\n✅ داده‌ها آماده شدند:")
print(f"   - تعداد نمونه‌ها: {n_samples}")
print(f"   - تعداد ویژگی‌ها: {n_features}")
print(f"   - تعداد کلاس‌ها: {n_classes}")


📊 Target (Churn): [1 1 0 1 0 0 1 1 1 0 0 0 1 1 1 1 0 1 1 0 1 0 0 0 1 1 1 1 0 1]
📈 توزیع کلاس‌ها: [12 18]
📊 درصد هر کلاس: Class 0: 40.0%, Class 1: 60.0%

🔧 Features: ['Age', 'LastTransaction', 'Gender_female', 'Gender_male', 'Payment Method_cash', 'Payment Method_cheque', 'Payment Method_credit card']
📊 شکل X: (30, 7)
📊 شکل y: (30,)

📋 نمونه از X (اولین 3 سطر):
   سطر 0: [ 38 171   0   1   0   1   0]
   سطر 1: [ 24 140   1   0   0   1   0]
   سطر 2: [82 39  0  1  1  0  0]

📋 نمونه از y (اولین 10 مقدار):
   [1 1 0 1 0 0 1 1 1 0]

✅ داده‌ها آماده شدند:
   - تعداد نمونه‌ها: 30
   - تعداد ویژگی‌ها: 7
   - تعداد کلاس‌ها: 2


In [8]:
# سلول چهارم – وارد کردن ماژول‌های مدل و معیارها

print("📦 در حال import ماژول‌های مدل و معیارها...")

from model.weighted_decision_tree import WeightedDecisionTreeModel
from custom_tree_classifier import CustomDecisionTreeClassifier
from utils.voting_split_manager import FNWeightedSplitManager
from criteria.gini import gini_criterion
from criteria.qg import gain_ratio_criterion  
from criteria.twoing import twoing_criterion
from criteria.ng import normalized_gain_criterion
from criteria.mch import multi_class_hellinger
from criteria.marsh import marsh_criterion
from criteria.gs import g_statistic_criterion
from criteria.dkm import dkm_criterion
from criteria.cs import chi_squared_criterion
from criteria.bhy import bhattacharyya_criterion
from criteria.ks import kolmogorov_smirnov_criterion

criteria_list = [
    "gini_criterion", "gain_ratio_criterion", "twoing_criterion",
    "normalized_gain_criterion", "multi_class_hellinger", "marsh_criterion", 
    "g_statistic_criterion", "dkm_criterion", "chi_squared_criterion",
    "bhattacharyya_criterion", "kolmogorov_smirnov_criterion"
]

print(f"✅ همه ماژول‌ها import شدند. تعداد معیارها: {len(criteria_list)}")


📦 در حال import ماژول‌های مدل و معیارها...
✅ همه ماژول‌ها import شدند. تعداد معیارها: 11


In [9]:
# سلول پنجم – ساخت مدل

print("🚀 ساخت مدل WeightedDecisionTreeModel...")

# ساخت مدل با داده‌های آماده
model = WeightedDecisionTreeModel(X_data=X, max_depth=5)

print("✅ مدل ساخته شد!")


🚀 ساخت مدل WeightedDecisionTreeModel...
✅ مدل ساخته شد!


In [10]:
# سلول ششم – آموزش مدل

print("🔥 آموزش مدل...")

# آموزش مدل با وزن‌دهی داینامیک فعال
model.fit(X, y)

print("🎉 مدل آموزش داده شد!")


🔥 آموزش مدل...
شروع آموزش مدل - شکل داده: X=(30, 7), y=(30,)
محاسبه وزن‌های اولیه پویا با معیارهای منفرد...
[INIT] شروع محاسبه وزن‌های اولیه...
[INIT] محاسبه FN برای معیار: gini
[INIT] gini: FN = 0
[INIT] محاسبه FN برای معیار: gain_ratio
[INIT] gain_ratio: FN = 0
[INIT] محاسبه FN برای معیار: twoing
[INIT] twoing: FN = 0
[INIT] محاسبه FN برای معیار: normalized_gain
[INIT] normalized_gain: FN = 0
[INIT] محاسبه FN برای معیار: multi_class_hellinger
[INIT] multi_class_hellinger: FN = 0
[INIT] محاسبه FN برای معیار: marshall
[INIT] marshall: FN = 0
[INIT] محاسبه FN برای معیار: g_statistic
[INIT] g_statistic: FN = 0
[INIT] محاسبه FN برای معیار: dkm
[INIT] dkm: FN = 0
[INIT] محاسبه FN برای معیار: chi_squared
[INIT] chi_squared: FN = 0
[INIT] محاسبه FN برای معیار: bhattacharyya
[INIT] bhattacharyya: FN = 0
[INIT] محاسبه FN برای معیار: kolmogorov_smirnov
[INIT] kolmogorov_smirnov: FN = 0
[INIT] gini: وزن نهایی = 0.0909
[INIT] gain_ratio: وزن نهایی = 0.0909
[INIT] twoing: وزن نهایی = 0.0909
[INIT]

KeyboardInterrupt: 

In [11]:
# سلول هفتم – پیش‌بینی و ارزیابی مدل

print("🔮 پیش‌بینی و ارزیابی مدل...")

# پیش‌بینی کلاس‌ها
predictions = model.predict(X)
print("✅ پیش‌بینی کلاس‌ها انجام شد")

# پیش‌بینی احتمالات
probabilities = model.predict_proba(X)
print("✅ پیش‌بینی احتمالات انجام شد")

# محاسبه دقت
accuracy = accuracy_score(y, predictions)
print(f"\n🎯 دقت مدل: {accuracy:.3f} ({accuracy*100:.1f}%)")

# ماتریس درهم‌ریختگی
cm = confusion_matrix(y, predictions)
print("\n📋 ماتریس درهم‌ریختگی:")
print(cm)

# استخراج TN, FP, FN, TP
tn, fp, fn, tp = cm.ravel()
print(f"\n📈 جزئیات:")
print(f"   True Positive (TP): {tp}")
print(f"   False Positive (FP): {fp}")
print(f"   True Negative (TN): {tn}")
print(f"   False Negative (FN): {fn}")


🔮 پیش‌بینی و ارزیابی مدل...
✅ پیش‌بینی کلاس‌ها انجام شد
✅ پیش‌بینی احتمالات انجام شد

🎯 دقت مدل: 0.800 (80.0%)

📋 ماتریس درهم‌ریختگی:
[[12  0]
 [ 6 12]]

📈 جزئیات:
   True Positive (TP): 12
   False Positive (FP): 0
   True Negative (TN): 12
   False Negative (FN): 6
